# Big Data project A.Y. 2024-2025

## Members

- Giovanni Antonioni
- Luca Rubboli - 0001083742

## Second job

In [1]:
import org.apache.spark
import org.apache.spark.sql.functions._
import org.apache.spark.sql.SparkSession

Intitializing Scala interpreter ...

Spark Web UI available at http://172.20.10.4:4040
SparkContext available as 'sc' (version = 3.5.1, master = local[*], app id = local-1744708735441)
SparkSession available as 'spark'


import org.apache.spark
import org.apache.spark.sql.functions._
import org.apache.spark.sql.SparkSession


In [7]:
// DO NOT EXECUTE - this is needed just to avoid showing errors in the following cells
 sc = spark.SparkContext.getOrCreate()

sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@595c127b


In [14]:
val spark = SparkSession.builder
.appName("First job")
.getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@439ea0c1


# Load Dataset
Define a custom function for loading the parquet files

In [35]:
import org.apache.spark.sql.DataFrame

def dfFromParquetFolder(folderPath: String): DataFrame = spark.read
    .option("recursiveFileLookup", "true")
    .parquet(folderPath)


import org.apache.spark.sql.DataFrame
dfFromParquetFolder: (folderPath: String)org.apache.spark.sql.DataFrame


We set here the path to the files

In [31]:
val datasetFolder = "../../../dataset/"
val yellowCab = s"$datasetFolder/yellow_cab/"
val greenCab = s"$datasetFolder/green_cab/"


datasetFolder: String = ../../../dataset/
yellowCab: String = ../../../dataset//yellow_cab/
greenCab: String = ../../../dataset//green_cab/


# Cleanup

Now we start to cleanup the various dataset that are present in our project.

In [36]:
val dfYellowCab = dfFromParquetFolder(yellowCab)
val dfDroppedNa = dfYellowCab.na.drop()

dfYellowCab: org.apache.spark.sql.DataFrame = [VendorID: int, tpep_pickup_datetime: timestamp_ntz ... 17 more fields]
dfDroppedNa: org.apache.spark.sql.DataFrame = [VendorID: int, tpep_pickup_datetime: timestamp_ntz ... 17 more fields]


In [37]:
dfDroppedNa.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.